# **Do High-SAT NYC Public High Schools Receive Higher Instructional Quality Ratings?**

### Why I Chose This Question

New York City’s public school system is huge and incredibly diverse, with schools varying widely in academic outcomes and school culture. Standardized test results like SAT scores are often used to judge school quality, but do they actually align with how students and families *experience* those schools?

I wanted to explore whether high academic performance is reflected in broader measures of instructional quality. If schools with strong SAT outcomes also earn high instructional ratings, it might suggest consistency between performance metrics and school environment. If not, it could indicate that academic scores alone don’t tell the full story.

This project uses two public datasets from NYC Open Data—one capturing SAT performance and another measuring school survey-based quality ratings—to explore that relationship.

---

### What Data I Used

I combined two datasets from the NYC Department of Education:

- **2012 SAT Results Dataset** — provides school-level SAT performance (Reading, Math, Writing)  
- **2019–20 School Quality Guide (High Schools)** — includes survey-based instructional quality ratings (`survey_pp_RI`)

The key that links these datasets is the **DBN (District–Borough–Number)**, a unique identifier for each school. Since the datasets come from different years, this analysis is not causal—it simply examines whether schools that historically perform well academically also receive high-quality ratings later on.

**Sources**

New York City Department of Education. (2021). *2012 SAT Results.* NYC Open Data.  
Retrieved from: https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4/about_data

New York City Department of Education. (2024). *2019–20 School Quality Guide: High Schools.* NYC Open Data.  
Retrieved from: https://data.cityofnewyork.us/Education/2019-20-School-Quality-Guide-High-Schools/ci36-d7ea/about_data

---

### How I Approached the Analysis

The workflow for this project looked like this:

1. Loaded both datasets into pandas.
2. Selected relevant fields: SAT scores, school names, enrollment numbers, instructional quality scores.
3. Converted SAT score columns to numeric values and computed a combined SAT total.
4. Merged the datasets using `dbn` as a common key.
5. Cleaned missing values (e.g., filling enrollment gaps with zero).
6. Created a scatterplot with an OLS trendline to visualize the relationship.
7. Calculated the correlation between SAT performance and instructional quality.

This helped me align academic outcomes with perceived instructional quality across NYC high schools.


## Methodology Summary

1. Load both datasets into pandas.
2. Select relevant variables related to SAT performance and instructional quality.
3. Convert score fields to numeric values and compute total SAT scores.
4. Merge datasets using `dbn` as the join key.
5. Remove or impute missing values (fill missing enrollment with zero).
6. Visualize the relationship using a scatter plot with an OLS trendline.
7. Calculate correlation to quantify the relationship statistically.


Step 1: Load Data

In [81]:
import pandas as pd
import plotly.express as px
from IPython.display import HTML

quality_path = "2019-20_School_Quality_Guide_High_Schools_20251121.csv"
sat_path = "2012_SAT_Results_20251121.csv"

df_quality = pd.read_csv(quality_path)
df_sat = pd.read_csv(sat_path)
df_quality.head(), df_sat.head()



(      dbn                                        school_name  school_type  \
 0  01M292                         Orchard Collegiate Academy  High School   
 1  01M448                University Neighborhood High School  High School   
 2  01M450                         East Side Community School  High School   
 3  01M509                            Marta Valle High School  High School   
 4  01M539  New Explorations into Science, Technology and ...  High School   
 
   enrollment survey_pp_RI survey_pp_CT survey_pp_SE survey_pp_ES survey_pp_SF  \
 0        226         0.74          0.8         0.71         0.88         0.86   
 1        492         0.85         0.91         0.87         0.92         0.92   
 2        364      No Data      No Data      No Data            1         0.97   
 3         97          0.8         0.93         0.81         0.89         0.98   
 4        654          0.8         0.79         0.74         0.82         0.92   
 
   survey_pp_TR  ...          QR_4_1

Step 2: Clean School Quality Data

In [75]:
# select relevant columns
df_quality_small = df_quality[["dbn", "school_name", "enrollment", "survey_pp_RI"]].copy()

# convert numeric fields
df_quality_small["enrollment"] = pd.to_numeric(df_quality_small["enrollment"], errors="coerce")
df_quality_small["survey_pp_RI"] = pd.to_numeric(df_quality_small["survey_pp_RI"], errors="coerce")

df_quality_small.head()



,dbn,school_name,enrollment,survey_pp_RI
0,01M292,Orchard Collegiate Academy,226.0,0.74
1,01M448,University Neighborhood High School,492.0,0.85
2,01M450,East Side Community School,364.0,NaN
3,01M509,Marta Valle High School,97.0,0.80
4,01M539,"New Explorations into Science, Technology and ...",654.0,0.80


Step 3: Clean SAT Data

In [76]:
df_sat_small = df_sat[[
    "DBN",
    "SCHOOL NAME",
    "Num of SAT Test Takers",
    "SAT Critical Reading Avg. Score",
    "SAT Math Avg. Score",
    "SAT Writing Avg. Score"
]].copy()

# rename DBN for merge
df_sat_small = df_sat_small.rename(columns={"DBN": "dbn"})

# convert numeric columns
score_cols = [
    "SAT Critical Reading Avg. Score",
    "SAT Math Avg. Score",
    "SAT Writing Avg. Score"
]

for col in score_cols:
    df_sat_small[col] = pd.to_numeric(df_sat_small[col], errors="coerce")

# compute total SAT score
df_sat_small["sat_total"] = (
    df_sat_small["SAT Critical Reading Avg. Score"]
    + df_sat_small["SAT Math Avg. Score"]
    + df_sat_small["SAT Writing Avg. Score"]
)

df_sat_small.head()


,dbn,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_total
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0


Step 4: Merge datasets

In [77]:
df_merged = pd.merge(df_quality_small, df_sat_small, on="dbn", how="inner")
df_merged.shape


(352, 10)

Step 5: Handle Missing Values

In [78]:
df_merged["enrollment"] = df_merged["enrollment"].fillna(0)
df_plot = df_merged.dropna(subset=["sat_total", "survey_pp_RI"])


Step 6: Visualization

In [83]:
fig = px.scatter(
    df_plot,
    x="sat_total",
    y="survey_pp_RI",
    hover_name="school_name",
    trendline="ols",
    color_discrete_sequence=["#1f77b4"],   # 单色
    labels={
        "sat_total": "Total SAT Score (Out of 2400)",
        "survey_pp_RI": "Instruction Quality Rating (Survey)",
    },
    title="NYC High Schools: SAT Scores vs Instructional Quality Ratings"
)

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))


Step 7: Correlation Analysis

In [ ]:
df_plot[["sat_total", "survey_pp_RI"]].corr()


,sat_total,survey_pp_RI
sat_total,1.000000,0.171178
survey_pp_RI,0.171178,1.000000


## Takeaways

The analysis finds only a weak positive relationship between SAT performance and instructional quality among NYC public high schools. Although schools with higher total SAT scores tend to have slightly higher “Rigorous Instruction” ratings, the correlation is low (≈ 0.17), and substantial variation exists across schools regardless of test outcomes.

This suggests that standardized academic performance does not strongly reflect survey-based measures of instructional quality. Instructional effectiveness appears to be influenced by broader factors such as curriculum design, teacher collaboration, school climate, and student support systems—dimensions that are not captured by SAT results alone.

Overall, SAT scores can indicate academic performance but should not be used as a standalone proxy for evaluating instructional quality or school effectiveness.
